# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [2]:
# your code goes here
import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'

df = pd.read_csv(url)

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

In [3]:
df['response'].unique()

array(['No', 'Yes', nan], dtype=object)

In [4]:
# Filtrar el DataFrame para incluir solo los clientes que tengan un total_claim_amount > 1000 y que hayan respondido 'Yes'
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]

# Mostrar las primeras filas del nuevo DataFrame para verificar
filtered_df.head()





,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


In [5]:
filtered_df2 = df[df['response'] == 'Yes']

In [6]:
grouped = filtered_df2.groupby(['policy_type', 'gender'])['total_claim_amount'].mean()



In [7]:
grouped.head()

policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
Name: total_claim_amount, dtype: float64

In [8]:
df['customer'].duplicated().any()

True

In [9]:
grouped3 = df.groupby(['state'])['customer'].nunique()

In [10]:
grouped3.head()

state
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
Name: customer, dtype: int64

In [11]:
type(grouped3)

pandas.core.series.Series

In [12]:
# Filtrar la Serie por los valores mayores que 1000
filtered_serie = grouped3[grouped3  > 500]

# Mostrar el resultado
print(filtered_serie)


state
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
Name: customer, dtype: int64


In [13]:
df.groupby(['education','gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()




,education,gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


El valor de vida del cliente máximo es mayor en mujeres en todos lo sniveles de educacion, menos para las personoas con nivel de estudios de High School o por debajo. El nivel máximo de sueldo está en las mujeres con Master.

El minimo es batante parecido tanto para mujeres como para hombres, pero cabe destacar que el más bajo es el de Bachelors y no el de High School o menores, siguiendole de cerca el nivel de College para mujeres.

La media más alta de sueldo es entre los hombre con High School y la media mas baja es entre las mujeres con nivel de estudios de Doctoras.

In [17]:
#Bonus: 5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are 
#arranged as columns and the states are arranged as rows.

In [25]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
df['effective_to_date']


0       2011-02-18
1       2011-01-18
2       2011-02-10
3       2011-01-11
4       2011-01-17
           ...    
10905   2011-01-19
10906   2011-01-06
10907   2011-02-06
10908   2011-02-13
10909   2011-01-08
Name: effective_to_date, Length: 10910, dtype: datetime64[ns]

In [27]:
df['month'] = df['effective_to_date'].dt.month
df['month'].unique()

array([2, 1])

In [29]:
import calendar
def numero_a_mes(mes):
    return calendar.month_name[mes]

# Aplicar la función a la columna y crear una nueva columna
df['month'] = df['month'].apply(numero_a_mes)

In [30]:
# Crear la tabla pivote con el conteo de policies por estado y mes
pivot_table = df.pivot_table(index='state', columns='month', values='number_of_policies', aggfunc='count', fill_value=0)

# Mostrar la tabla pivote
print(pivot_table)


month       February  January
state                        
Arizona          929     1008
California      1634     1918
Nevada           442      551
Oregon          1344     1565
Washington       425      463
